In [318]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import io, openpyxl, requests

In [319]:
url = "https://en.wikipedia.org/wiki/List_of_Namma_Metro_stations"
headers = {
    "User-Agent": "WikiTablesFetcher/1.0 (contact: ms@thecontrarian.in)",
    "Accept-Language": "en-US,en;q=0.9"
}

resp = requests.get(url, headers=headers, timeout=30)
resp.raise_for_status()

NammaMetro_Stations = pd.read_html(io.StringIO(resp.text), flavor="lxml", displayed_only=True)

NammaMetro_Stations_1 = NammaMetro_Stations[1].loc[:, [('Line', 'Line'), 
                                                       ('Abbreviations', 'Abbreviations'), 
                                                       ('Station name', 'English'), 
                                                       ('Station name', 'Kannada')]].copy()
NammaMetro_Stations_4 = NammaMetro_Stations[4].loc[:, [('Line', 'Line'), 
                                                       ('Abbreviations', 'Abbreviations'), 
                                                       ('Station name', 'English'), 
                                                       ('Station name', 'Kannada')]].copy()
NammaMetro_Stations = pd.concat([NammaMetro_Stations_1, NammaMetro_Stations_4], ignore_index=True)

NammaMetro_Stations.columns = [f'{a}' for a,b in NammaMetro_Stations.columns]
NammaMetro_Stations.columns = ['line_name', 'code', 'name_eng', 'name_kan']
NammaMetro_Stations['line_name'] = NammaMetro_Stations['line_name'].str.split().str[0]
NammaMetro_Stations['code'] = NammaMetro_Stations['code'].str.replace('TBC', '[TBC]')

line_colours = {'Purple': '🟣',  'Green': '🟢', 'Yellow': '🟡', 'Blue': '🔵', 
                'Pink'  : '🌸', 'Orange': '🟠',    'Red': '🔴', 'Grey': '⚪'}
NammaMetro_Stations['line_colour'] = NammaMetro_Stations['line_name'].map(line_colours)

lines = NammaMetro_Stations.groupby(['name_eng'])['line_colour'].sum().copy()
NammaMetro_Stations['lines'] = NammaMetro_Stations['name_eng'].map(lines)
NammaMetro_Stations = NammaMetro_Stations[['code', 'name_eng', 'name_kan', 'lines']].drop_duplicates()
NammaMetro_Stations['is_terminus'] = NammaMetro_Stations['name_eng'].str.contains('\\*')
NammaMetro_Stations['is_interchange'] = NammaMetro_Stations['name_eng'].str.contains('†')
NammaMetro_Stations['name_eng'] = NammaMetro_Stations['name_eng'].str.split('†').str[0].str.split('*').str[0].str.strip()

NammaMetro_Stations = NammaMetro_Stations.sort_values('name_eng').reset_index(drop=True)
display(NammaMetro_Stations)

,code,name_eng,name_kan,lines,is_terminus,is_interchange
0,[TBC],Agara,ಅಗರ,🔵🔴,False,True
1,[TBC],Airport City,ವಿಮಾನ ನಿಲ್ದಾಣ ನಗರ,🔵,False,False
2,[TBC],Ambedkar Nagar,ಅಂಬೇಡ್ಕರ್ ನಗರ,🔴,False,False
3,AGPP,Attiguppe,ಅತ್ತಿಗುಪ್ಪೆ,🟣,False,False
4,[TBC],BEL Circle,ಬಿ ಇ ಎಲ್ ವೃತ್ತ,🟠,False,False
...,...,...,...,...,...,...
167,[TBC],Vinayaka Nagar,ವಿನಾಯಕ ನಗರ,⚪,False,False
168,WHTM,Whitefield (Kadugodi),ವೈಟ್ ಫೀಲ್ಡ್ (ಕಾಡುಗೋಡಿ),🟣,True,False
169,PUTH,Yelachenahalli,ಯಲಚೇನಹಳ್ಳಿ,🟢,False,False
170,[TBC],Yelahanka,ಯಲಹಂಕ,🔵,False,False


In [320]:
print(f"Number of functioning stations: {len(NammaMetro_Stations[~(NammaMetro_Stations['code'] == '[TBC]')])}")
print(f"Number of stations under construction: {len(NammaMetro_Stations[(NammaMetro_Stations['code'] == '[TBC]')])}")
for line in line_colours.values():
    print(f"Number of {line} stations: {len(NammaMetro_Stations[(NammaMetro_Stations['lines'].str.contains(line))])}")

columns = ['code', 'name_eng', 'name_kan', 'lines']

print("\nTERMINUS STATIONS")
display(NammaMetro_Stations[columns][NammaMetro_Stations['is_terminus']].sort_index())

print("INTERCHANGE STATIONS")
display(NammaMetro_Stations[columns][NammaMetro_Stations['is_interchange']].sort_index())

Number of functioning stations: 83
Number of stations under construction: 89
Number of 🟣 stations: 37
Number of 🟢 stations: 32
Number of 🟡 stations: 16
Number of 🔵 stations: 29
Number of 🌸 stations: 18
Number of 🟠 stations: 19
Number of 🔴 stations: 28
Number of ⚪ stations: 8

TERMINUS STATIONS


,code,name_eng,name_kan,lines
23,CLGA,Challaghatta,ಚಲ್ಲಘಟ್ಟ,🟣
31,BMSD,Delta Electronics Bommasandra,ಡೆಲ್ಟಾ ಎಲೆಕ್ಟ್ರಾನಿಕ್ಸ್ ಬೊಮ್ಮಸಂದ್ರ,🟡
50,[TBC],Hebbala,ಹೆಬ್ಬಾಳ,🔵🟠🔴
65,[TBC],JP Nagar 4th Phase,ಜಯಪ್ರಕಾಶ ನಗರ 4ನೇ ಹಂತ,🌸🟠
75,[TBC],KIAL Terminals,ಕೆಂಪೇಗೌಡ ಅಂತಾರಾಷ್ಟ್ರೀಯ ವಿಮಾನ ನಿಲ್ದಾಣ,🔵
82,[TBC],Kalena Agrahara,ಕಾಳೇನ ಅಗ್ರಹಾರ,🌸
87,[TBC],Kempapura,ಕೆಂಪಾಪುರ,🔵🟠
105,BIET,Madavara,ಮಾದಾವರ,🟢
122,[TBC],Nagawara,ನಾಗವಾರ,🌸🔵
137,RVR,Rashtreeya Vidyalaya Road,ರಾಷ್ಟ್ರೀಯ ವಿದ್ಯಾಲಯ ರಸ್ತೆ,🟢🟡


INTERCHANGE STATIONS


,code,name_eng,name_kan,lines
0,[TBC],Agara,ಅಗರ,🔵🔴
22,SBJT,Central Silk Board,ಕೇಂದ್ರ ರೇಷ್ಮೆ ಮಂಡಳಿ,🟡🔵
28,[TBC],Dairy Circle,ಡೈರಿ ವೃತ್ತ,🌸🔴
50,[TBC],Hebbala,ಹೆಬ್ಬಾಳ,🔵🟠🔴
62,[TBC],Ibbaluru,ಇಬ್ಬಲೂರು,🔵🔴
65,[TBC],JP Nagar 4th Phase,ಜಯಪ್ರಕಾಶ ನಗರ 4ನೇ ಹಂತ,🌸🟠
71,JDHP,Jayadeva Hospital,ಜಯದೇವ ಆಸ್ಪತ್ರೆ,🟡🌸
87,[TBC],Kempapura,ಕೆಂಪಾಪುರ,🔵🟠
117,KGWA,"Nadaprabhu Kempegowda Station, Majestic","ನಾಡಪ್ರಭು ಕೆಂಪೇಗೌಡ ನಿಲ್ದಾಣ, ಮೆಜೆಸ್ಟಿಕ್",🟣🟢
122,[TBC],Nagawara,ನಾಗವಾರ,🌸🔵


In [321]:
for line in line_colours.values():
    display(NammaMetro_Stations[NammaMetro_Stations['lines'].str.contains(line)])

,code,name_eng,name_kan,lines,is_terminus,is_interchange
3,AGPP,Attiguppe,ಅತ್ತಿಗುಪ್ಪೆ,🟣,False,False
9,BYPL,Baiyappanahalli,ಬೈಯ್ಯಪ್ಪನಹಳ್ಳಿ,🟣,False,False
15,JTPM,Benniganahalli,ಬೆನ್ನಿಗಾನಹಳ್ಳಿ,🟣,False,False
23,CLGA,Challaghatta,ಚಲ್ಲಘಟ್ಟ,🟣,True,False
27,CBPK,Cubbon Park,ಕಬ್ಬನ್ ಪಾರ್ಕ್,🟣,False,False
30,DJNR,Deepanjali Nagar,ದೀಪಾಂಜಲಿ ನಗರ,🟣,False,False
38,VDSA,"Dr. B. R. Ambedkar Station, Vidhana Soudha","ಡಾ. ಬಿ. ಆರ್. ಅಂಬೇಡ್ಕರ್ ನಿಲ್ದಾಣ, ವಿಧಾನ ಸೌಧ",🟣,False,False
44,GDCP,Garudacharpalya,ಗರುಡಾಚಾರ್‍‍ಪಾಳ್ಯ,🟣,False,False
49,HLRU,Halasuru,ಹಲಸೂರು,🟣,False,False
53,DKIA,Hoodi,ಹೂಡಿ,🟣,False,False


,code,name_eng,name_kan,lines,is_terminus,is_interchange
10,BSNK,Banashankari,ಬನಶಂಕರಿ,🟢,False,False
24,CKPE,Chickpete,ಚಿಕ್ಕಪೇಟೆ,🟢,False,False
25,CKBD,Chikkabidarakallu,ಚಿಕ್ಕಬಿದರಕಲ್ಲು,🟢,False,False
29,DSH,Dasarahalli,ದಾಸರಹಳ್ಳಿ,🟢,False,False
33,KLPK,Doddakallasandra,ದೊಡ್ಡಕಲ್ಲಸಂದ್ರ,🟢,False,False
45,YPI,Goraguntepalya,ಗೊರಗುಂಟೆಪಾಳ್ಯ,🟢,False,False
69,JLHL,Jalahalli,ಜಾಲಹಳ್ಳಿ,🟢,False,False
70,JPN,Jaya Prakash Nagar,ಜಯಪ್ರಕಾಶ ನಗರ,🟢,False,False
72,JYN,Jayanagar,ಜಯನಗರ,🟢,False,False
93,APRC,Konanakunte Cross,ಕೋಣನಕುಂಟೆ ಕ್ರಾಸ್,🟢,False,False


,code,name_eng,name_kan,lines,is_terminus,is_interchange
5,BTML,BTM Layout,ಬಿ ಟಿ ಎಂ ಬಡಾವಣೆ,🟡,False,False
16,HOSR,Beratena Agrahara,ಬೆರಟೇನ ಅಗ್ರಹಾರ,🟡,False,False
17,HBGI,Biocon Hebbagodi,ಬಯೋಕಾನ್ ಹೆಬ್ಬಗೋಡಿ,🟡,False,False
18,HSRL,Bommanahalli,ಬೊಮ್ಮನಹಳ್ಳಿ,🟡,False,False
22,SBJT,Central Silk Board,ಕೇಂದ್ರ ರೇಷ್ಮೆ ಮಂಡಳಿ,🟡🔵,False,True
31,BMSD,Delta Electronics Bommasandra,ಡೆಲ್ಟಾ ಎಲೆಕ್ಟ್ರಾನಿಕ್ಸ್ ಬೊಮ್ಮಸಂದ್ರ,🟡,True,False
39,ETCT,Electronic City,ಎಲೆಕ್ಟ್ರಾನಿಕ್ ಸಿಟಿ,🟡,False,False
52,OFDC,Hongasandra,ಹೊಂಗಸಂದ್ರ,🟡,False,False
56,BSRD,Hosa Road,ಹೊಸ ರಸ್ತೆ,🟡,False,False
59,HSKR,Huskur Road,ಹುಸ್ಕೂರು ರಸ್ತೆ,🟡,False,False


,code,name_eng,name_kan,lines,is_terminus,is_interchange
0,[TBC],Agara,ಅಗರ,🔵🔴,False,True
1,[TBC],Airport City,ವಿಮಾನ ನಿಲ್ದಾಣ ನಗರ,🔵,False,False
6,[TBC],Bagalur Cross,ಬಾಗಲೂರು ಕ್ರಾಸ್,🔵,False,False
7,[TBC],Bagmane Developers - DRDO Sports Complex,ರ.ಸಂ.ಅ.ಸಂ. ಕ್ರೀಡಾ ಸಂಕೀರ್ಣ,🔵,False,False
12,[TBC],Bellandur,ಬೆಳ್ಳಂದೂರು,🔵,False,False
22,SBJT,Central Silk Board,ಕೇಂದ್ರ ರೇಷ್ಮೆ ಮಂಡಳಿ,🟡🔵,False,True
32,[TBC],Doddajala,ದೊಡ್ಡಜಾಲ,🔵,False,False
35,[TBC],Doddanekundi,ದೊಡ್ಡನೆಕ್ಕುಂದಿ,🔵,False,False
40,[TBC],Embassy TechVillage Kadubeesanahalli,ಕಾಡುಬೀಸನಹಳ್ಳಿ,🔵,False,False
46,[TBC],HBR Layout,ಎಚ್‌ ಬಿ ಆರ್ ಬಡಾವಣೆ,🔵,False,False


,code,name_eng,name_kan,lines,is_terminus,is_interchange
20,[TBC],Cantonment Railway Station,ದಂಡು ರೈಲ್ವೇ ನಿಲ್ದಾಣ,🌸,False,False
28,[TBC],Dairy Circle,ಡೈರಿ ವೃತ್ತ,🌸🔴,False,True
58,[TBC],Hulimavu,ಹುಳಿಮಾವು,🌸,False,False
60,[TBC],IIMB,ಐ ಐ ಎಂ ಬಿ,🌸,False,False
65,[TBC],JP Nagar 4th Phase,ಜಯಪ್ರಕಾಶ ನಗರ 4ನೇ ಹಂತ,🌸🟠,True,True
71,JDHP,Jayadeva Hospital,ಜಯದೇವ ಆಸ್ಪತ್ರೆ,🟡🌸,False,True
80,[TBC],Kadugundanahalli,ಕಾಡುಗುಂಡನಹಳ್ಳಿ,🌸,False,False
82,[TBC],Kalena Agrahara,ಕಾಳೇನ ಅಗ್ರಹಾರ,🌸,True,False
102,[TBC],Lakkasandra,ಲಕ್ಕಸಂದ್ರ,🌸,False,False
104,[TBC],Langford Town,ಲ್ಯಾಂಗ್ಫೋರ್ಡ್ ಟೌನ್,🌸,False,False


,code,name_eng,name_kan,lines,is_terminus,is_interchange
4,[TBC],BEL Circle,ಬಿ ಇ ಎಲ್ ವೃತ್ತ,🟠,False,False
8,[TBC],Bahubhali Nagara,ಬಾಹುಬಲಿ ನಗರ,🟠,False,False
26,[TBC],Chowdeshwari Nagara,ಚೌಡೇಶ್ವರಿ ನಗರ,🟠,False,False
37,[TBC],Dr. Ambedkar Institute of Technology,ಡಾ||ಅಂಬೇಡ್ಕರ್‌ ತಾಂತ್ರಿಕ ಮಹಾವಿದ್ಯಾಲಯ,🟠,False,False
42,[TBC],Freedom Fighters Cross,ಫ್ರೀಡಂ ಫೈಟರ್ಸ್ ಕ್ರಾಸ್,🟠,False,False
50,[TBC],Hebbala,ಹೆಬ್ಬಾಳ,🔵🟠🔴,True,True
57,[TBC],Hosakerahalli Cross,ಹೊಸಕೆರೆಹಳ್ಳಿ ಕ್ರಾಸ್,🟠,False,False
65,[TBC],JP Nagar 4th Phase,ಜಯಪ್ರಕಾಶ ನಗರ 4ನೇ ಹಂತ,🌸🟠,True,True
66,[TBC],JP Nagar 5th Phase,ಜೆಪಿ ನಗರ ೫ನೇ ಹಂತ,🟠,False,False
78,[TBC],Kadirenahalli Cross,ಕದಿರೇನಹಳ್ಳಿ ಕ್ರಾಸ್,🟠,False,False


,code,name_eng,name_kan,lines,is_terminus,is_interchange
0,[TBC],Agara,ಅಗರ,🔵🔴,False,True
2,[TBC],Ambedkar Nagar,ಅಂಬೇಡ್ಕರ್ ನಗರ,🔴,False,False
11,[TBC],Basaveshwara Circle,ಬಸವೇಶ್ವರ ವೃತ್ತ,🔴,False,False
13,[TBC],Bellandur Gate,ಬೆಳ್ಳಂದೂರು ಗೇಟ್,🔴,False,False
14,[TBC],Bengaluru Golf Course,ಬೆಂಗಳೂರು ಗಾಲ್ಫ್ ಕೋರ್ಸ್,🔴,False,False
21,[TBC],Carmelaram,ಕಾರ್ಮೆಲರಾಮ್,🔴,False,False
28,[TBC],Dairy Circle,ಡೈರಿ ವೃತ್ತ,🌸🔴,False,True
34,[TBC],Doddakannelli,ದೊಡ್ಡಕನ್ನೆಲ್ಲಿ,🔴,False,False
36,[TBC],Dommasandra,ದೊಮ್ಮಸಂದ್ರ,🔴,False,False
43,[TBC],Ganganagar,ಗಂಗಾನಗರ,🔴,False,False


,code,name_eng,name_kan,lines,is_terminus,is_interchange
19,[TBC],Byadarahalli,ಬ್ಯಾಡರಹಳ್ಳಿ,⚪,False,False
41,[TBC],Forest Gate,ಅರಣ್ಯ ದ್ವಾರ,⚪,False,False
51,[TBC],Herohalli,ಹೇರೋಹಳ್ಳಿ,⚪,False,False
74,[TBC],KHB Colony,ಕೆ ಎಚ್ ಬಿ ಕಾಲೋನಿ,⚪,False,False
77,[TBC],Kadabagere,ಕಡಬಗೆರೆ,⚪,False,False
153,[TBC],Sumanahalli Cross,ಸುಮನಹಳ್ಳಿ ಕ್ರಾಸ್,🟠⚪,False,False
154,[TBC],Sunkadakatte,ಸುಂಕದಕಟ್ಟೆ,⚪,False,False
167,[TBC],Vinayaka Nagar,ವಿನಾಯಕ ನಗರ,⚪,False,False


_____


In [322]:
df_hourly = pd.read_excel('/Users/home/DEV/MY PROJECTS/namma-metro-ridership-tracker/hourly-ridership/raw/station-hourly.xlsx')
df_hourly

,BUSINESS DATE,STATION,00:00 Hrs To 01:00 Hrs,01:00 Hrs To 02:00 Hrs,02:00 Hrs To 03:00 Hrs,03:00 Hrs To 04:00 Hrs,04:00 Hrs To 05:00 Hrs,05:00 Hrs To 06:00 Hrs,06:00 Hrs To 07:00 Hrs,07:00 Hrs To 08:00 Hrs,...,15:00 Hrs To 16:00 Hrs,16:00 Hrs To 17:00 Hrs,17:00 Hrs To 18:00 Hrs,18:00 Hrs To 19:00 Hrs,19:00 Hrs To 20:00 Hrs,20:00 Hrs To 21:00 Hrs,21:00 Hrs To 22:00 Hrs,22:00 Hrs To 23:00 Hrs,23:00 Hrs To Last train,TOTAL
0,2025-08-01,11-Baiyappanahalli,0,0,0,0,19,83,215,918,...,456,497,880,942,923,489,373,134,15,13836
1,2025-08-01,12-SV Road,0,0,0,0,7,39,85,334,...,573,866,1078,1068,856,433,211,132,26,9067
2,2025-08-01,13-Indiranagar,0,0,0,0,4,41,175,621,...,1301,1660,2303,3456,2982,1866,1248,712,161,24471
3,2025-08-01,14-Halasuru,0,0,0,0,5,51,192,570,...,418,387,566,756,660,580,317,151,17,9924
4,2025-08-01,15-Trinity,0,0,0,0,2,17,50,158,...,537,838,1899,3404,2572,1093,421,319,38,13746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1350,2025-08-18,311-Electronic City,0,0,0,0,0,15,29,85,...,207,242,235,261,178,113,95,43,0,2761
1351,2025-08-18,312-Infosys Foundation Konappana Agrahara,0,0,0,0,10,35,129,263,...,346,474,594,574,411,282,165,45,0,5534
1352,2025-08-18,313-Huskur Road,0,0,0,0,0,6,53,115,...,107,171,147,207,128,77,51,37,0,1954
1353,2025-08-18,314-Biocon Hebbagodi,0,0,0,0,0,5,37,83,...,73,93,113,138,80,59,25,15,0,1461


In [323]:
station_name_fixes = {
    'SV Road': 'Swami Vivekananda Road',
    'MG Road': 'Mahatma Gandhi Road',
    'Vidhana Soudha': 'Dr. B. R. Ambedkar Station, Vidhana Soudha',
    'Sir.M. Visveshwaraya': 'Sir.M.Visvesvaraya Stn., Central College',
    'Kempegowda': 'Nadaprabhu Kempegowda Station, Majestic',
    'Bangalore City Station': 'Krantivira Sangolli Rayanna Railway Station',
    'Hosahalli': 'Sri Balagangadharanatha Swamiji Station, Hosahalli',
    'Nayandahalli': 'Pantharapalya–Nayandahalli',
    'Pattangere': 'Pattanagere',
    'Krishnarajapura': 'Krishnarajapura (K.R.Pura)',
    'Seetharampalya': 'Seetharamapalya',
    'Nallur Halli': 'Nallurhalli',
    'Whitefield (Kadugodi)': 'Whitefield (Kadugodi)',
    'Yeshwanthpur Industry': 'Goraguntepalya',
    'Kuvempu Road': 'Mahakavi Kuvempu Road',
    'KR Market': 'Krishna Rajendra Market',
    'RV Road': 'Rashtreeya Vidyalaya Road',
    'Banashankarl': 'Banashankari',
    'JP Nagar': 'Jaya Prakash Nagar',
    'Puttenahalli': 'Yelachenahalli',
    'Thalghattapura': 'Thalaghattapura',
    'Silk Institute': 'Silk Institute',
    'Manjunatha Nagar': 'Manjunathanagara',
}

In [324]:
for col in df_hourly.columns.tolist()[2:-1]:
    col_reformatted = col.split()[0][:2]
    df_hourly[col_reformatted] = df_hourly[col]

df_hourly['date'] = pd.to_datetime(df_hourly['BUSINESS DATE'], format='%Y-%m-%d')
df_hourly['station_name'] = df_hourly['STATION'].str.split('-').str[1]
df_hourly['station_code'] = df_hourly['STATION'].str.split('-').str[0]

df_hourly = df_hourly[['date', 'station_code', 'station_name'] + [f"{n:02d}" for n in range(24)] + ['TOTAL']].copy()
df_hourly['total_check'] = df_hourly['TOTAL'] == df_hourly[[f"{n:02d}" for n in range(24)]].sum(axis=1)
df_hourly = df_hourly.loc[:, :'TOTAL'] if df_hourly.loc[:, 'total_check'].isna().sum() == 0 else df_hourly

df_hourly['station_name'] = df_hourly['station_name'].replace(station_name_fixes)

condition = dict(zip(NammaMetro_Stations['name_eng'], NammaMetro_Stations['code']))
df_hourly['station_code'] = df_hourly['station_name'].map(condition)

df_hourly

,date,station_code,station_name,00,01,02,03,04,05,06,...,15,16,17,18,19,20,21,22,23,TOTAL
0,2025-08-01,BYPL,Baiyappanahalli,0,0,0,0,19,83,215,...,456,497,880,942,923,489,373,134,15,13836
1,2025-08-01,SVRD,Swami Vivekananda Road,0,0,0,0,7,39,85,...,573,866,1078,1068,856,433,211,132,26,9067
2,2025-08-01,IDN,Indiranagar,0,0,0,0,4,41,175,...,1301,1660,2303,3456,2982,1866,1248,712,161,24471
3,2025-08-01,HLRU,Halasuru,0,0,0,0,5,51,192,...,418,387,566,756,660,580,317,151,17,9924
4,2025-08-01,TTY,Trinity,0,0,0,0,2,17,50,...,537,838,1899,3404,2572,1093,421,319,38,13746
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1350,2025-08-18,ETCT,Electronic City,0,0,0,0,0,15,29,...,207,242,235,261,178,113,95,43,0,2761
1351,2025-08-18,ECTN,Infosys Foundation Konappana Agrahara,0,0,0,0,10,35,129,...,346,474,594,574,411,282,165,45,0,5534
1352,2025-08-18,HSKR,Huskur Road,0,0,0,0,0,6,53,...,107,171,147,207,128,77,51,37,0,1954
1353,2025-08-18,HBGI,Biocon Hebbagodi,0,0,0,0,0,5,37,...,73,93,113,138,80,59,25,15,0,1461


`Record Date,Smart Cards,NCMC,Tokens,QR,Group Ticket,Total`

`2025-08-01,425980,25105,200670,180952,157,832864`

`2025-08-02,300912,17623,248287,217745,414,784981`

`2025-08-04,443237,26260,218087,180895,107,868586`

`2025-08-06,475439,29037,213525,179785,185,897971`

`2025-08-11,503837,32198,303165,208382,450,1048032`

`2025-08-12,525613,34320,263412,208766,576,1032687`

`2025-08-14,488569,33052,293707,268138,322,1083788`

`2025-08-15,194028,14440,386683,144182,396,739729`

`2025-08-16,271190,19134,334983,121204,133,746644`

`2025-08-18,485345,34029,270054,228711,532,1018671`

In [361]:
df_pairhourly = pd.read_excel('./hourly-ridership/raw/stationpair-hourly.xlsx')

station_code_fixes = {
    'BNSK': 'BSNK',
    'BRCS': 'SRCS',
    'BYPH': 'BYPL',
    'CHPK': 'CKPE',
    'CLG': 'CLGA',
    'CLGT': 'CLGA',
    'JAYN': 'JYN',
    'JHLI': 'JLHL',
    'JIDL': 'CKBD',
    'KRMA': 'KRAM',
    'KRPM': 'KRAM',
    'LAGH': 'LBGH',
    'MGRD': 'MAGR',
    'PATC': 'PATG',
    'RVRD': 'RVR',
    'VJNR': 'VJN',
}

df_pairhourly['STATION'] = df_pairhourly['STATION'].map(station_code_fixes).fillna(df_pairhourly['STATION'])

df_pairhourly = df_pairhourly.rename(columns={"BUSINESS DATE": "date", "STATION": "station_code"})
df_pairhourly = df_pairhourly.rename(columns=station_code_fixes)

df_pairhourly['hour'] = df_pairhourly['date'].str.split('-').str[3].str[:2].astype(int)
df_pairhourly['date'] = df_pairhourly['date'].str.split('-').str[0:3].str.join('-')
df_pairhourly['date'] = pd.to_datetime(df_pairhourly['date'], format='%Y-%m-%d')

df_pairhourly['lines'] = df_pairhourly['station_code'].map(NammaMetro_Stations[NammaMetro_Stations['code']!='[TBC]'].set_index('code')['lines']).copy()

df_pairhourly.dropna(subset=['station_code'], inplace=True)
df_pairhourly = df_pairhourly[['date', 'hour', 'station_code', 'lines']+df_pairhourly.columns[2:-2].tolist()]

In [364]:
inspection_date = '2025-08-11'
inspection_hour = 15

with pd.option_context("display.max_rows", None, "display.max_columns", None): 
    display(df_pairhourly[(df_pairhourly['date'] == inspection_date) & (df_pairhourly['hour'] == inspection_hour)])

,date,hour,station_code,lines,BYPL,SVRD,IDN,HLRU,TTY,MAGR,CBPK,VDSA,VSWA,KGWA,SRCS,MIRD,HSLI,VJN,AGPP,DJNR,MYRD,NYHM,RRRN,BGUC,PATG,MLSD,KGIT,CLGA,JTPM,KRAM,MDVP,GDCP,DKIA,VWIA,KDNH,VDHP,SSHP,ITPL,KDGD,UWVL,WHTM,NGSA,DSH,JLHL,PYID,PEYA,YPI,YPM,SSFY,MHLI,RJNR,KVPR,SPRU,SPGD,CKPE,KRMT,NLC,LBGH,SECE,JYN,RVR,BSNK,JPN,PUTH,APRC,KLPK,VJRH,TGTP,APTS,MNJN,CKBD,BIET,RGDT,JDHP,BTML,SBJT,HSRL,OFDC,MSRN,CKBR,BSRD,HOSR,ETCT,ECTN,HSKR,HBGI,BMSD
14801,2025-08-11,15,BYPL,🟣,0,9,31,10,8,13,11,7,25,23,12,5,9,2,3,13,0,0,8,2,2,8,2,2,7,37,7,2,2,3,3,2,10,11,1,0,6,9,2,2,1,0,1,8,3,1,7,4,2,3,8,7,8,8,3,0,4,8,3,4,5,5,1,1,1,1,0,2,0,0,0,0,0,0,0,0,1,0,0,1,1,0,0
14802,2025-08-11,15,SVRD,🟣,8,2,11,15,2,9,2,3,9,28,17,18,17,19,21,8,14,2,8,10,4,6,2,1,17,24,7,11,3,3,2,5,4,2,5,0,9,4,7,6,1,2,2,14,3,3,2,1,2,5,8,7,0,1,1,2,0,3,4,3,2,3,0,1,0,0,2,1,0,4,2,0,0,0,1,0,0,0,0,5,1,0,0
14803,2025-08-11,15,IDN,🟣,59,40,7,68,20,46,25,10,32,36,26,19,15,20,16,8,15,7,16,11,5,18,8,7,92,62,24,18,14,12,7,12,4,19,11,8,28,8,6,9,2,4,4,7,11,8,9,2,9,24,11,3,14,8,3,9,8,13,4,7,7,11,0,2,1,0,3,9,0,0,3,0,0,0,0,1,2,1,1,4,0,1,0
14804,2025-08-11,15,HLRU,🟣,24,29,17,2,8,41,15,4,14,15,9,7,8,1,3,1,1,1,2,1,2,2,3,5,53,34,11,11,2,2,2,2,1,6,3,3,6,3,2,6,0,0,0,4,1,3,4,1,1,10,8,2,3,2,3,2,2,3,2,1,1,0,2,0,2,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,1,0,1
14805,2025-08-11,15,TTY,🟣,14,5,16,10,2,22,11,10,19,22,12,10,13,7,9,5,6,2,4,3,2,6,8,4,39,18,9,3,5,4,3,3,2,6,3,0,6,6,6,7,2,0,3,1,5,0,8,4,3,8,0,0,7,4,2,4,3,5,1,5,4,4,1,0,3,0,3,4,1,1,0,0,0,0,0,0,0,0,0,2,1,0,1
14806,2025-08-11,15,MAGR,🟣🌸,49,18,68,108,19,8,28,14,30,62,35,13,10,24,26,8,8,6,15,9,5,13,8,10,73,30,9,6,16,10,11,9,1,7,10,6,23,4,8,5,5,1,8,3,15,8,7,4,7,22,7,3,5,1,4,17,12,6,11,8,16,5,2,2,6,0,1,4,0,1,2,1,1,0,0,2,2,1,1,1,0,0,1
14807,2025-08-11,15,CBPK,🟣,16,4,37,12,9,29,3,4,14,58,23,8,13,18,14,3,6,0,5,11,4,10,1,3,35,21,8,3,6,4,8,5,4,9,9,3,19,7,3,2,0,1,0,3,4,2,9,8,2,12,3,6,10,7,3,8,4,6,6,4,8,3,3,1,4,1,0,3,3,2,3,0,3,0,0,0,2,0,3,3,0,0,0
14808,2025-08-11,15,VDSA,🟣,8,10,42,45,15,20,11,7,88,88,43,22,23,31,33,9,5,1,5,14,1,21,11,10,57,15,5,2,4,3,1,5,0,0,10,4,27,2,10,2,2,0,4,10,1,8,9,9,5,13,5,13,10,21,4,9,2,5,3,4,13,1,3,1,7,0,0,6,2,0,1,0,0,0,1,0,0,0,4,0,1,1,2
14809,2025-08-11,15,VSWA,🟣,35,19,21,7,26,33,24,105,11,70,38,19,52,51,44,6,9,9,13,16,3,14,14,1,55,11,10,4,8,2,1,3,1,1,10,4,17,12,14,4,3,1,5,13,8,10,16,12,9,20,3,5,11,8,6,5,8,3,8,5,6,4,3,3,4,4,2,7,0,2,0,0,0,0,3,0,2,0,1,1,0,2,9
14810,2025-08-11,15,KGWA,🟣🟢,33,19,27,20,19,32,9,26,31,3,23,27,20,43,35,26,5,3,10,17,13,22,11,26,73,40,13,8,18,15,9,15,9,11,7,10,28,13,45,18,1,2,13,59,35,16,21,8,17,27,38,24,31,23,15,37,20,50,28,15,37,5,11,11,21,4,11,21,3,11,15,5,8,2,19,4,12,3,22,8,6,7,18


In [395]:
inspection_date = '2025-08-11'

# (df_pairhourly['lines'].str.contains('🟣')) & 

ct_df = df_pairhourly[(df_pairhourly['date'] == inspection_date)].copy()

ct_df['Total'] = ct_df.loc[:, ct_df.columns[4:]].sum(axis=1)
ct_df['Total'].sum()

np.int64(793560)

In [ ]:
ct_df.pivot_table(index='station_code', columns=ct_df.columns[4:], values='ridership', aggfunc='sum')